(solph-linear)=

# Linear solph heat pump model

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def load_temperature_data():
    """Returns temperature (in °C) over timestamp
    """
    # weather data from https://wetterstation.physik.rwth-aachen.de/datenbank.php
    df = pd.read_csv("../2023_Stundenstatistik.txt", delim_whitespace=True)
    df.index = (pd.to_datetime(
        [
            f"{year}-{month}-{day} {hour}:00:00"
            for year, month, day, hour in zip(df['#Jahr'], df['Monat'], df['Tag'], df['Stunde'])
        ]
    ))
    df = df[["T_Mid"]]
    df.rename(columns={"T_Mid": "Ambient temperature (°C)"}, inplace=True)

    return df


input_data = load_temperature_data().head(24*3)

# A crude demand model should be enough for us. Let the demand be 500 W per K below 15 °C.
input_data["Heat load (kW)"] = (0.5 * (15 - input_data["Ambient temperature (°C)"])).clip(lower=0)

plt.plot(input_data["Ambient temperature (d°C)"]/10, "b-")
plt.ylabel("Ambient temperature (°C)").set_color("blue")

plt.twinx()

plt.plot(input_data["simple TESPy COP"], "r-")
plt.ylabel("COP").set_color("red")

plt.show()


Now, we want to add an air-source heat-pump centered heat supply system.  Short online research for example yields:

* Rated heating output (kW) at operating point A7/W35: 9.1 kW
* Coefficient of performance ε (COP) A7/W35: 4.9

(https://www.viessmann.co.uk/en/products/heat-pump/vitocal-150a.html,  type 150.A16)

In [ ]:
datasheet_cop = 4.9
carnot_cop_7_35 = (35+273.15) / (35-7)
cpf_7_35 = datasheet_cop / carnot_cop_7_35

carnot_cop_2_40 = (40+273.15) / (40-2)
cpf_2_40 = datasheet_cop / carnot_cop_2_40

input_data["cpf COP 7 -> 35"] = cpf_7_35 * (35+273.15) / (35 - input_data["Ambient temperature (°C)"])
input_data["cpf COP 2 -> 40"] = cpf_2_40 * (40+273.15) / (40 - input_data["Ambient temperature (°C)"] + 5)

In [ ]:
plt.plot(input_data["Ambient temperature (°C)"], "b-")
plt.ylabel("Ambient temperature (°C)").set_color("blue")

plt.twinx()

plt.plot(input_data["cpf COP 7 -> 35"], "r-")
plt.plot(input_data["cpf COP 2 -> 40"], "r--")
plt.ylabel("COP").set_color("red")

plt.show()

In [ ]:
hp_thermal_power = 9.1  # kW
cop = input_data["constant efficiency COP"][:-1]
demand = input_data["Heat load (kW)"][:-1]

In [ ]:
import oemof.solph as solph

es = solph.EnergySystem(timeindex=input_data.index, infer_last_interval=False)

b_electricity = solph.Bus(label="electricity")
b_heat_35C = solph.Bus(label="heat 35C")
es.add(b_electricity, b_heat_35C)

electricity_grid = solph.components.Source(
    label="electricity grid",
    outputs={b_electricity: solph.Flow(variable_costs=0.4)},  # €/kWh
)

heat_demand = solph.components.Sink(
    label="heat demand",
    inputs={b_heat_35C: solph.Flow(nominal_value=1, fix=demand)},  # kW
)
es.add(electricity_grid, heat_demand)


heat_pump = solph.components.Transformer(
    label="heat pump",
    inputs={b_electricity: solph.Flow()},
    outputs={b_heat_35C: solph.Flow(nominal_value=hp_thermal_power)},
    conversion_factors={
        b_electricity: 1 / cop,
        b_heat_35C: 1,
    },
)

es.add(heat_pump)

es.add(solph.components.Transformer(
    label="heating rod",
    inputs={b_electricity: solph.Flow()},
    outputs={b_heat_35C: solph.Flow()},
))

thermal_storage = solph.components.GenericStorage(
    label='thermal_storage',
    inputs={b_heat_35C: solph.Flow()},
    outputs={b_heat_35C: solph.Flow()},
    nominal_storage_capacity=8.7,  # Assume 5 k of spread and 1.5 m³ volume 
)

es.add(thermal_storage)

model = solph.Model(energysystem=es)

model.solve()

results = solph.processing.results(model)



In [ ]:
heat_supply = results[(heat_pump, b_heat_35C)]["sequences"]["flow"]

plt.plot(np.sort(heat_supply)[::-1], "r-", label="heat supply")
plt.plot(np.sort(demand)[::-1], "b-", label="heat demand")
plt.ylabel("Power (kW)")
plt.grid()
plt.legend()

electricity_consumption = float(results[(electricity_grid, b_electricity)]["sequences"].sum())
print(f"Electricity demand: {electricity_consumption:.1f} kWh")

plt.show()
